<a href="https://colab.research.google.com/github/eddieguo-1128/honors/blob/main/Thesis_Code_Part3_Bio_Clinical_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thesis Code Part 3: Bio_Clinical_BERT

In [3]:
# Import Packages
import pandas as pd
import numpy as np

In [4]:
# Connect to drive and acquire data from Part 1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/SP23/Honors Thesis/DATA H195B/repo/honors

/content/drive/My Drive/SP23/Honors Thesis/DATA H195B/repo/honors


In [6]:
reports_df = pd.read_csv('reports_df.csv',index_col=0)
pd.options.display.max_colwidth = 0
reports_df

,id,findings,impression,full-text,label
0,162,"Heart size normal. Lungs are clear. XXXX are normal. No pneumonia, effusions, edema, pneumothorax, adenopathy, nodules or masses.",Normal chest,"Heart size normal. Lungs are clear. XXXX are normal. No pneumonia, effusions, edema, pneumothorax, adenopathy, nodules or masses. Normal chest",normal
1,1390,"The heart size and pulmonary vascularity appear within normal limits. Lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. VP shunt tubing is identified. The bony structures, as visualized, appear unremarkable.",No evidence of active disease.,"The heart size and pulmonary vascularity appear within normal limits. Lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. VP shunt tubing is identified. The bony structures, as visualized, appear unremarkable. No evidence of active disease.",abnormal
2,604,The heart size and pulmonary vascularity appear within normal limits. The lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. PICC line is in XXXX. The tip is in the upper right atrium.,1. No evidence of active disease.,The heart size and pulmonary vascularity appear within normal limits. The lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. PICC line is in XXXX. The tip is in the upper right atrium. 1. No evidence of active disease.,abnormal
3,2699,"The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion. Multiple punctate round XXXX XXXX over the abdomen on the lateral view. These may reside within, or outside of the patient.",No acute cardiopulmonary disease.,"The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion. Multiple punctate round XXXX XXXX over the abdomen on the lateral view. These may reside within, or outside of the patient. No acute cardiopulmonary disease.",normal
4,2841,"The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion.",No acute cardiopulmonary disease.,"The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion. No acute cardiopulmonary disease.",normal
...,...,...,...,...,...
3950,2694,"The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion.",No acute cardiopulmonary disease.,"The heart is normal in size and contour. The lungs are clear, without evidence of infiltrate. There is no pneumothorax or effusion. No acute cardiopulmonary disease.",normal
3951,2858,Heart size normal. Tortuous aorta. Sequela primary granulomatous disease. Lungs clear. Minimal spurring in the thoracic spine.,No acute cardiopulmonary finding.,Heart size normal. Tortuous aorta. Sequela primary granulomatous disease. Lungs clear. Minimal spurring in the thoracic spine. No acute cardiopulmonary finding.,abnormal
3952,2680,The lungs are clear. There is no pleural effusion or pneumothorax. The heart and mediastinum are normal. The skeletal structures are normal.,No acute pulmonary disease.,The lungs are clear. There is no pleural effusion or pneumothorax. The heart and mediastinum are normal. The skeletal structures are normal. No acute pulmonary disease.,normal
3953,85,There are scattered calcified granulomas. No focal infiltrate. No pleural effusion or pneumothorax. Heart size and mediastinal contour are within normal limits. There are degenerative changes of the spine.,No evidence of active disease.,There are scattered calcified granulomas. No focal infiltrate. No pleural effusion or pneumothorax. Heart size and mediastinal contour are within normal limits. There are degenerative changes of the spine. No evidence of active disease.,abnormal


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.7 MB/s eta 0:00:00


In [8]:
from sklearn.model_selection import train_test_split

# fix seed for reproducibility
np.random.seed(2023)

train, test = train_test_split(reports_df, test_size = 0.2, shuffle=True)
train, validation = train_test_split(train, test_size = 0.1, shuffle=True)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# load the pre-trained Bio Clinical BERT model
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)

# prepare the dataset
train_texts = list(train['full-text'].values)
train_labels = [0 if x=="normal" else 1 for x in train.label.values]
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# fine-tune the model
optimizer = AdamW(model.parameters(), lr=5e-5)
for epoch in range(3):
    for inputs, masks, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# evaluate the model
model.eval()
eval_texts = list(validation['full-text'].values)
eval_labels = [0 if x=="normal" else 1 for x in validation.label.values]
eval_encodings = tokenizer(eval_texts, padding=True, truncation=True, return_tensors='pt')
eval_dataset = TensorDataset(eval_encodings['input_ids'], eval_encodings['attention_mask'], torch.tensor(eval_labels))
eval_dataloader = DataLoader(eval_dataset, batch_size=16)
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, masks, labels in eval_dataloader:
        outputs = model(inputs, attention_mask=masks)
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {correct / total}')


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model